In [1]:
import os
import shutil
import warnings
import urllib.request as request
from contextlib import closing

import cupy

import dask_cudf
import dask_ml

from cuml.manifold import UMAP as cuUMAP
from cuml.dask.decomposition import PCA as cuDaskPCA
from cuml.dask.cluster import KMeans as cuDaskKMeans
from cuml.dask.manifold import UMAP as cuDaskUMAP

from dask.distributed import Client, LocalCluster
from dask_cuda import initialize, LocalCUDACluster
from dask_cuda.local_cuda_cluster import cuda_visible_devices

from bokeh.io.export import export_png
from bokeh.plotting import figure
from bokeh.models.tickers import FixedTicker
from bokeh.io import output_notebook, push_notebook, show

from nvidia.cheminformatics.chembldata import ChEmblData

warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

output_notebook()

Loading BokehJS ...

### Configurations and settings

In [2]:
# Please add or remove device ids that can be used
# CUDA_VISIBLE_DEVICES=[0]
CUDA_VISIBLE_DEVICES = cuda_visible_devices(0).split(',')

pca_comps = 64
n_clusters = 6
n_neighbors=100
num_cols=100000

enable_tcp_over_ucx = True
enable_nvlink = False
enable_infiniband = False

COLORS = ["#406278", "#e32636", "#9966cc", "#cd9575", "#915c83", "#008000",
          "#ff9966", "#848482", "#8a2be2", "#de5d83", "#800020", "#e97451",
          "#5f9ea0", "#36454f", "#008b8b", "#e9692c", "#f0b98d", "#ef9708",
          "#0fcfc0", "#9cded6", "#d5eae7", "#f3e1eb", "#f6c4e1", "#f79cd4"]

### Functions

In [3]:
def show_cluster_plot(ldcudf, title='UMAP'):
    """
    Draws a scatter plots from output of UMAP.
    """
    umap_fig = figure(title=title, width=800, output_backend="webgl")
    clusters = ldcudf['cluster'].unique().compute().values_host

    for cluster in clusters:
        query = 'cluster == %s' % (cluster)

        cdf = ldcudf.query(query)
        cdf = cdf.compute()

        if cdf.shape[0] == 0:
            continue

        x_array = cupy.fromDlpack(cdf['x'].to_dlpack())
        y_array = cupy.fromDlpack(cdf['y'].to_dlpack())

        umap_fig.circle(x_array.get(),
                        y_array.get(),
                        size=2,
                        color=COLORS[cluster],
                        alpha=0.5, legend = 'Cluster ' + str(cluster))

    umap_fig.legend.location = 'top_right'
    umap_fig.legend.title = 'Clusters'
    
    umap_fig_handle = show(umap_fig, notebook_handle=True)
    push_notebook(handle=umap_fig_handle)
    

def recluster(tcdf):
    """
    Recluster on a dataset
    """
    tcdf = tcdf.persist()

    print('Reclustering...')
    tcdf = tcdf.drop(['x', 'y', 'cluster'], axis=1)

    print('Computing KMEANS...')
    kmeans_float = cuDaskKMeans(client=client, n_clusters=n_clusters)
    kmeans_float.fit(tcdf)
    kmeans_labels = kmeans_float.predict(tcdf)

    print('Computing UMAP...')
    local_model = cuUMAP()
    X_train = tcdf.compute()
    local_model.fit(X_train)

    umap_model = cuDaskUMAP(local_model,
                            n_neighbors=n_neighbors,
                            a=1.0,
                            b=1.0,
                            learning_rate=1.0,
                            client=client)
    Xt = umap_model.transform(tcdf)

    # Add back the column required for plotting and to correlating data
    # between re-clustering
    tcdf['x'] = Xt[0]
    tcdf['y'] = Xt[1]
    tcdf['cluster'] = kmeans_labels
    
    return tcdf

# Download ChEMBL database

In [4]:
data_dir = "/data/db"
db_file = os.path.join(data_dir, 'chembl_27.db')

if not os.path.exists(db_file):
    print('Downloading ChEMBL db...')

    os.makedirs(data_dir, exist_ok=True)
    with closing(request.urlopen('ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_27_sqlite.tar.gz')) as r:
        with open(db_file, 'wb') as f:
            shutil.copyfileobj(r, f)

    print('Download completed')
else:
    print('Reusing available ChEMBL db at', db_file)

Reusing available ChEMBL db at /data/db/chembl_27.db


In [5]:
cluster = LocalCUDACluster(protocol="ucx",
                           dashboard_address=':9001',
                           # TODO: automate visible device list
                           CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)
client = Client(cluster)
client

Client Scheduler: ucx://127.0.0.1:40284 Dashboard: http://127.0.0.1:9001/status,Cluster Workers: 8 Cores: 8 Memory: 540.95 GB


# Generate fingerprint from ChEMBL

The 4 in ECFP4 corresponds to the diameter of the atom environments considered, while the Morgan fingerprints take a radius parameter. So a Morgan fingerprint with radius=2 is roughly equivalent to ECFP4 and FCFP4.

In [6]:
%%time
chem_data = ChEmblData(db_file=db_file)

# Fetch and generate Finger prints
ddf = chem_data.fetch_all_props(num_recs=num_cols)

dcudf = dask_cudf.from_dask_dataframe(ddf)
dcudf = dcudf.persist()

pca = cuDaskPCA(client=client, n_components=pca_comps)
dcudf = pca.fit_transform(dcudf)

CPU times: user 2.68 s, sys: 864 ms, total: 3.54 s
Wall time: 23.4 s


# Clustering

In [7]:
%%time

kmeans_float = cuDaskKMeans(client=client, n_clusters=n_clusters)
kmeans_float.fit(dcudf)
kmeans_labels = kmeans_float.predict(dcudf)

CPU times: user 100 ms, sys: 20 ms, total: 120 ms
Wall time: 419 ms


In [8]:
%%time
local_model = cuUMAP()
X_train = dcudf.compute()
local_model.fit(X_train)

umap_model = cuDaskUMAP(local_model,
                        n_neighbors=n_neighbors,
                        a=1.0,
                        b=1.0,
                        learning_rate=1.0,
                        client=client)
Xt = umap_model.transform(dcudf)

dcudf['x'] = Xt[0]
dcudf['y'] = Xt[1]
dcudf['cluster'] = kmeans_labels

dcudf.head(5)

CPU times: user 988 ms, sys: 824 ms, total: 1.81 s
Wall time: 2.35 s


,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,x,y,cluster
0,-0.865109,-0.292492,0.786789,-0.095175,0.796129,0.656859,-0.295244,-0.621802,0.742635,0.438073,...,-0.029014,-1.098615,-0.170794,0.309797,-0.376544,0.184059,0.118639,0.166541,4.795009,4
1,-1.204017,0.119189,-0.013176,0.055445,0.797985,0.426475,-0.399370,-0.880231,0.117580,-0.019904,...,0.135545,-0.742777,0.154685,0.313335,0.162623,0.287277,0.228658,0.203778,4.708547,4
2,-0.850133,-0.352980,-0.017702,-0.182518,1.496359,-0.199614,0.545056,-0.290458,0.719915,0.412472,...,0.180975,-0.345707,-0.168005,0.314767,0.036302,0.189138,-0.078456,0.138080,4.907867,3
3,-1.142298,-0.016493,-0.014621,-0.310373,0.534076,0.251173,-0.598150,-0.777292,0.082535,-0.105265,...,0.202079,-0.818716,0.555176,-0.028419,0.068724,0.366090,0.147236,0.237351,4.583308,4
4,-1.133956,-0.142514,0.030310,-0.095415,1.681546,0.710921,-0.142584,-0.484768,1.036501,0.604438,...,0.309328,-0.765515,-0.198358,0.188346,-0.081390,0.270140,-0.011902,0.158487,4.844131,4


In [9]:
%%time
show_cluster_plot(dcudf)

CPU times: user 2.74 s, sys: 560 ms, total: 3.3 s
Wall time: 6.69 s


# Re-clustering

In [12]:
%%time
# Filter data before reclustering
cluster = 1                                         # <- PLEASE CHANGE CLUSTER AS REQUIRED
query = 'cluster == ' + str(cluster)
tcdf = dcudf.query(query)

show_cluster_plot(tcdf, title='UMAP - cluster ' + str(cluster))
tcdf = recluster(tcdf)
print('Plotting UMAP...')
show_cluster_plot(tcdf, title='UMAP - cluster ' + str(cluster))

Reclustering...
Computing KMEANS...
Computing UMAP...
Plotting UMAP...


CPU times: user 2.69 s, sys: 632 ms, total: 3.32 s
Wall time: 9.61 s
